In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Data Import
data = pd.read_csv("/home/tjselevani/Desktop/Apps/vscode/python/python analysis/data/last-3-months-transactions.csv")

# Convert 'created_at' column to datetime if not already
data['created_at'] = pd.to_datetime(data['created_at'])

# Extract time in minutes since start of the day
data.loc[:, 'minutes'] = data['created_at'].dt.hour * 60 + data['created_at'].dt.minute

# Extract time in hours since start of the day
data.loc[:, 'hours'] = data['created_at'].dt.hour

# Extract the day of the week (0 = Monday, 6 = Sunday)
data.loc[:, 'day_of_week'] = data['created_at'].dt.dayofweek

# Extract the week from 'created_at'
data.loc[:, 'week'] = data['created_at'].dt.to_period('W').astype(str)

# Extract the month from 'created_at'
data.loc[:, 'month'] = data['created_at'].dt.to_period('M').astype(str)

# Assuming your data import and initial processing is already done
# Starting from where you have the data loaded and time fields extracted

# 1. MONTHLY ANALYSIS BY VEHICLE
# ---------------------------------

# Aggregate data by month for each vehicle
monthly_by_vehicle = data.groupby(['month', 'vehicle_booked'])['amount'].sum().reset_index()

# Pivot the data to have vehicles as columns
monthly_pivot = monthly_by_vehicle.pivot(index='month', columns='vehicle_booked', values='amount').reset_index()

# Sort by month (assuming format is YYYY-MM)
monthly_pivot = monthly_pivot.sort_values('month')

# VISUALIZATION 1: Line chart for monthly earnings by vehicle
plt.figure(figsize=(14, 7))
for vehicle in vehicles:
    if vehicle in monthly_pivot.columns:
        plt.plot(monthly_pivot['month'], monthly_pivot[vehicle], marker='o', linewidth=2, label=vehicle)

plt.title("Monthly Earnings by Vehicle", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Total Earnings (KSH)", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(title="Vehicles")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("monthly_earnings_line.png", dpi=300)
plt.show()

# VISUALIZATION 2: Bar chart for monthly earnings by vehicle
# Reshape data for grouped bar chart
monthly_melt = monthly_by_vehicle.copy()

plt.figure(figsize=(14, 7))
sns.barplot(x='month', y='amount', hue='vehicle_booked', data=monthly_melt, palette='viridis')

plt.title("Monthly Earnings by Vehicle", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Total Earnings (KSH)", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title="Vehicles")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("monthly_earnings_bar.png", dpi=300)
plt.show()

# 2. TOTAL EARNINGS ACROSS ALL VEHICLES
# --------------------------------------

# Weekly totals for all vehicles combined
weekly_total = data.groupby('week')['amount'].sum().reset_index()
weekly_total = weekly_total.sort_values('week')

# Monthly totals for all vehicles combined
monthly_total = data.groupby('month')['amount'].sum().reset_index()
monthly_total = monthly_total.sort_values('month')

# Yearly totals (extract year from month)
data['year'] = pd.to_datetime(data['created_at']).dt.year
yearly_total = data.groupby('year')['amount'].sum().reset_index()

# VISUALIZATION 3: Weekly trends for all vehicles
plt.figure(figsize=(14, 7))
sns.lineplot(x='week', y='amount', data=weekly_total, marker='o', color='blue', linewidth=2)

plt.title("Weekly Total Earnings - All Vehicles", fontsize=16)
plt.xlabel("Week", fontsize=12)
plt.ylabel("Total Earnings (KSH)", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(rotation=45)

# Add value annotations
for i, row in weekly_total.iterrows():
    plt.text(i, row['amount'], f"{int(row['amount']):,}", ha='center', va='bottom', fontsize=8, rotation=45)

plt.tight_layout()
plt.savefig("weekly_total_earnings.png", dpi=300)
plt.show()

# VISUALIZATION 4: Monthly trends for all vehicles
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='month', y='amount', data=monthly_total, color='skyblue')

plt.title("Monthly Total Earnings - All Vehicles", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Total Earnings (KSH)", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45)

# Add value annotations
for i, row in monthly_total.iterrows():
    ax.text(i, row['amount'], f"{int(row['amount']):,}", ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig("monthly_total_earnings.png", dpi=300)
plt.show()

# VISUALIZATION 5: Yearly comparison for all vehicles
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='year', y='amount', data=yearly_total, palette='Blues_d')

plt.title("Yearly Total Earnings - All Vehicles", fontsize=16)
plt.xlabel("Year", fontsize=12)
plt.ylabel("Total Earnings (KSH)", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add value annotations
for i, row in yearly_total.iterrows():
    ax.text(i, row['amount'], f"{int(row['amount']):,}", ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig("yearly_total_earnings.png", dpi=300)
plt.show()

# VISUALIZATION 6: Combined view showing monthly breakdown by vehicle and total
plt.figure(figsize=(16, 8))

# Create a stacked bar chart
monthly_by_vehicle_pivot = monthly_by_vehicle.pivot_table(index='month', columns='vehicle_booked', values='amount')
monthly_by_vehicle_pivot = monthly_by_vehicle_pivot.fillna(0)  # Replace NaN with 0
monthly_by_vehicle_pivot = monthly_by_vehicle_pivot.sort_index()  # Sort by month

monthly_by_vehicle_pivot.plot(kind='bar', stacked=True, figsize=(16, 8), colormap='viridis')

plt.title("Monthly Earnings Breakdown by Vehicle", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Total Earnings (KSH)", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.legend(title="Vehicles", bbox_to_anchor=(1.05, 1), loc='upper left')

# Add total value annotations
monthly_totals = monthly_by_vehicle_pivot.sum(axis=1)
for i, total in enumerate(monthly_totals):
    plt.text(i, total + (total*0.01), f"Total: {int(total):,}", ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig("monthly_stacked_earnings.png", dpi=300)
plt.show()

NameError: name 'vehicles' is not defined

<Figure size 1400x700 with 0 Axes>